In [1]:
import numpy as np
import scipy.stats as stats
from scipy.stats.stats import pearsonr
import pandas as pd

#plotting modules

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

# Stats/Regresions Packages
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

# make sure charts appear in the notebook:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

For this project i am trying to look at the impact of presence of community health worker on the accurate prediction of Malaria incedences amongst households in Uganda. I will be using the DHS of Uganda. For this assignment I have given a preliminary look at the data, merged the data files

In [3]:
df = pd.read_stata('/Users/meyhar/Dropbox/2014-15/UGHR72DT/UGHR72FL.dta')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5345 entries, 0 to 5344
Columns: 2096 entries, hhid to sh228_7
dtypes: category(1786), float64(255), int16(7), int32(3), int8(19), object(26)
memory usage: 20.9+ MB


In [5]:
df.head()

,hhid,hv000,hv001,hv002,hv003,hv004,hv005,hv006,hv007,hv008,...,sh225_5,sh225_6,sh225_7,sh228_1,sh228_2,sh228_3,sh228_4,sh228_5,sh228_6,sh228_7
0,1 1,UG6,1,1,2,1,227246,12,2014,1380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1 3,UG6,1,3,1,1,227246,12,2014,1380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1 5,UG6,1,5,2,1,227246,12,2014,1380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1 6,UG6,1,6,1,1,227246,12,2014,1380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1 8,UG6,1,8,1,1,227246,12,2014,1380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.describe().T

/Users/meyhar/anaconda2/envs/econ/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,count,mean,std,min,25%,50%,75%,max
hv001,5345.0,1.057274e+02,60.994291,1.0,53.0,106.0,159.0,210.0
hv002,5345.0,5.159457e+01,44.725716,1.0,20.0,42.0,72.0,521.0
hv004,5345.0,1.057274e+02,60.994291,1.0,53.0,106.0,159.0,210.0
hv005,5345.0,1.000000e+06,555861.500000,106375.0,590788.0,923469.0,1260494.0,3542943.0
hv006,5345.0,7.984471e+00,5.290896,1.0,1.0,12.0,12.0,12.0
hv007,5345.0,2.014366e+03,0.481636,2014.0,2014.0,2014.0,2015.0,2015.0
hv008,5345.0,1.380371e+03,0.495077,1380.0,1380.0,1380.0,1381.0,1382.0
hv009,5345.0,5.152292e+00,2.944091,1.0,3.0,5.0,7.0,24.0
hv010,5345.0,1.027877e+00,0.819263,0.0,1.0,1.0,1.0,7.0
hv011,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df2 = pd.read_stata('/Users/meyhar/Dropbox/2014-15/UGPR72DT/UGPR72FL.dta')

In [8]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27539 entries, 0 to 27538
Columns: 302 entries, hhid to sh228
dtypes: category(238), float64(32), int16(7), int32(3), int8(19), object(3)
memory usage: 15.1+ MB


In [9]:
df2.head()

,hhid,hvidx,hv000,hv001,hv002,hv003,hv004,hv005,hv006,hv007,...,sh218f,sh218g,sh218h,sh218y,sh219,sh220,sh221,sh223a,sh225,sh228
0,1 1,1,UG6,1,1,2,1,227246,12,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1 1,2,UG6,1,1,2,1,227246,12,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1 1,3,UG6,1,1,2,1,227246,12,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1 1,4,UG6,1,1,2,1,227246,12,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1 1,5,UG6,1,1,2,1,227246,12,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
merge_1= pd.merge(df, df2,on= 'hhid', how='inner' )

In [11]:
merge_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27539 entries, 0 to 27538
Columns: 2397 entries, hhid to sh228
dtypes: float64(406), int16(14), int32(6), int64(6), int8(38), object(1927)
memory usage: 494.0+ MB


In [12]:
merge_1.tail()

,hhid,hv000_x,hv001_x,hv002_x,hv003_x,hv004_x,hv005_x,hv006_x,hv007_x,hv008_x,...,sh218f,sh218g,sh218h,sh218y,sh219,sh220,sh221,sh223a,sh225,sh228
27534,210 47,UG6,210,47,1,210,1171767,12,2014,1380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27535,210 47,UG6,210,47,1,210,1171767,12,2014,1380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27536,210 47,UG6,210,47,1,210,1171767,12,2014,1380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27537,210 47,UG6,210,47,1,210,1171767,12,2014,1380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27538,210 47,UG6,210,47,1,210,1171767,12,2014,1380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
merge_1.isnull().sum()

hhid           0
hv000_x        0
hv001_x        0
hv002_x        0
hv003_x        0
hv004_x        0
hv005_x        0
hv006_x        0
hv007_x        0
hv008_x        0
hv009_x        0
hv010_x        0
hv011_x    27539
hv012_x        0
hv013_x        0
hv014_x        0
hv015_x        0
hv016_x        0
hv017_x        0
hv018_x        0
hv019_x        0
hv020_x        0
hv021_x        0
hv022_x        0
hv023_x        0
hv024_x        0
hv025_x        0
hv026_x    27539
hv027_x        0
hv028_x        0
           ...  
sh128aa    27539
sh128ab    27539
sh128ac    27539
sh128ad    27539
sh128ae    27539
sh128af    27539
sh128ax    27539
sh128az    27539
hmla1       8056
hmlb1      12280
hmlc1      20420
hmld1      25847
h6bidx     22344
sh214      22383
sh216      24041
sh218a     22623
sh218b     22623
sh218c     22623
sh218d     22623
sh218e     22623
sh218f     22623
sh218g     22623
sh218h     22623
sh218y     22623
sh219      25945
sh220      26055
sh221      26122
sh223a     264

In [14]:
##Using the DHS of Uganda I am trying to look at the impact of presence of community health worker on prediction and treatment of preventable diseases.
##The use of predictive analysis here would be to predict how fast and correctly can households treat illnesses if they rely on CHW.
##A CHW is trained in basic primary health care in areas which lack proper medical facilities.

#The Disease I will be looking at is Malaria

In [15]:
#Deleting columns with no observations
del merge_1['hv026_x']

In [16]:
#del merge_1['hv031_x', 'hv041_x', 'hv044_x', 'hv202_x', 'hv234a_x','hv236_x','hv230a_x','hv230b_x','hv232_x','hv232b_x','hv237_x','hv232c_x','hv232d_x','hv232e_x','hv232y_x','hv234_x']

In [17]:
del merge_1['hv031_x']

In [18]:
del merge_1['hv041_x']

In [19]:
del merge_1['hv044_x']

In [20]:
del merge_1['hv202_x']

In [21]:
del merge_1['hv234a_x']

In [22]:
del merge_1['hv236_x']

In [23]:
del merge_1['hv230a_x']

In [24]:
del merge_1['hv230b_x']

In [31]:
del merge_1['hv232_x']

In [32]:
del merge_1['hv232b_x']

In [34]:
del merge_1['hv237_x']

In [35]:
del merge_1['hv234_x']